#Nutribi : Notebook for Training Model (using tensorflow lite model Maker)

## 1. Prerequisites


In [ ]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 593kB 7.6MB/s 
     |████████████████████████████████| 112kB 37.2MB/s 
     |████████████████████████████████| 645kB 22.2MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 174kB 34.8MB/s 
     |████████████████████████████████| 122kB 34.9MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 1.1MB 35.1MB/s 
     |████████████████████████████████| 849kB 36.1MB/s 
     |████████████████████████████████| 1.2MB 52.1MB/s 
     |████████████████████████████████| 6.3MB 53.2MB/s 
     |████████████████████████████████| 686kB 47.9MB/s 
     |████████████████████████████████| 194kB 50.1MB/s 
     |████████████████████████████████| 358kB 48.2MB/s 
     |████████████████████████████████| 38.2MB 68kB/s 
     |████████████████████████████████| 102kB 14.8MB/s 


Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
# from google.cloud import storage

# # Initialise a client
# storage_client = storage.Client("cloud-ml-data")
# # Create a bucket object for our bucket
# bucket = storage_client.get_bucket(bucket_name)
# # Create a blob object from the filepath
# blob = bucket.blob("/img/openimage/csv/salads_ml_use.cs")
# # Download the file to a destination
# blob.download_to_filename(destination_file_name)

## Train your salad detection model

In [ ]:
spec = model_spec.get('efficientdet_lite2')

**Load the dataset from google cloud storage.**

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('dataset_object_detection_final.csv')

In [ ]:
train_data

**Train the TensorFlow model with the training data.**

- Epoch : 50
- Batch Size : 8

Default set :
  - steps_per_execution = 1
  - moving_average_decay = 0
  - learning_rate: 0.001
  - gradient_norm: 3.7450

In [ ]:
model = object_detector.create(train_data,epochs=50,model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
59/59 [==============================] - 74s 703ms/step - det_loss: 1.6723 - cls_loss: 1.1793 - box_loss: 0.0099 - reg_l2_loss: 0.0770 - loss: 1.7493 - learning_rate: 0.0090 - gradient_norm: 1.6125 - val_det_loss: 1.5518 - val_cls_loss: 1.1791 - val_box_loss: 0.0075 - val_reg_l2_loss: 0.0770 - val_loss: 1.6288
Epoch 2/50
59/59 [==============================] - 36s 612ms/step - det_loss: 1.3086 - cls_loss: 0.9962 - box_loss: 0.0062 - reg_l2_loss: 0.0770 - loss: 1.3856 - learning_rate: 0.0100 - gradient_norm: 3.4574 - val_det_loss: 1.2274 - val_cls_loss: 0.9332 - val_box_loss: 0.0059 - val_reg_l2_loss: 0.0771 - val_loss: 1.3044
Epoch 3/50
59/59 [==============================] - 36s 621ms/step - det_loss: 1.0766 - cls_loss: 0.8208 - box_loss: 0.0051 - reg_l2_loss: 0.0772 - loss: 1.1538 - learning_rate: 0.0099 - gradient_norm: 4.0964 - val_det_loss: 1.1920 - val_cls_loss: 0.9019 - val_box_loss: 0.0058 - val_reg_l2_loss: 0.0773 - val_loss: 1.2693
Epoch 4/50
59/59 [=============

**Evaluate the model with the test data.**


In [ ]:
model.evaluate(test_data)

3/3 [==============================] - 24s 4s/step



{'AP': 0.2853088,
 'AP50': 0.40662488,
 'AP75': 0.3208054,
 'AP_/Banana': 0.11647411,
 'AP_/Cabbage': 0.4032958,
 'AP_/Carrot': 0.22689329,
 'AP_/Egg': 0.36905634,
 'AP_/Pasta': 0.6822422,
 'AP_/Potato': 0.20640133,
 'AP_/Shrimp': 0.039367117,
 'AP_/Tomato': 0.2387402,
 'APl': 0.3041484,
 'APm': 0.1136757,
 'APs': 0.020509195,
 'ARl': 0.5609739,
 'ARm': 0.26574075,
 'ARmax1': 0.29298848,
 'ARmax10': 0.47564918,
 'ARmax100': 0.5315923,
 'ARs': 0.07692308}

**Export as a TensorFlow Lite model.**


In [ ]:
model.export(export_dir='.')

**Evaluate the TensorFlow Lite model.**


In [ ]:
model.evaluate_tflite('model.tflite', test_data)

150/150 [==============================] - 1193s 8s/step



{'AP': 0.26149943,
 'AP50': 0.3813324,
 'AP75': 0.29211673,
 'AP_/Banana': 0.112677924,
 'AP_/Cabbage': 0.43915927,
 'AP_/Carrot': 0.20741428,
 'AP_/Egg': 0.2662298,
 'AP_/Pasta': 0.65255314,
 'AP_/Potato': 0.22007677,
 'AP_/Shrimp': 0.02151401,
 'AP_/Tomato': 0.17237021,
 'APl': 0.28227845,
 'APm': 0.035562817,
 'APs': 0.023762377,
 'ARl': 0.4838489,
 'ARm': 0.18981482,
 'ARmax1': 0.27585658,
 'ARmax10': 0.42055234,
 'ARmax100': 0.45392972,
 'ARs': 0.023076924}